# Einsum
Why?
- Extremely Convenient and Compact
- It can combine multiple tensor operations in one

Cons
- Can be confusing
- We might lose some performance
    - As it is not optimized as compared for a specific function call
    - Sometimes it can be faster as we can replace multiple function calls to one function call

<p align="center">
<img src="../images/Einsum.png" style="width:450px;height:250px;">
</p>

### Important Definitions
1. **Free indices**: Are the indices specified in the output
2. **Summation indices**: All other indices. Those that appear in the input argument and NOT in the output specification

E.g. 

i,j --> Free indices, 

k--> Summation index in the previous example

In [12]:
import numpy as np
import torch

In [3]:
A = np.random.rand(5)
B = np.random.rand(3)
A, B

(array([0.44730454, 0.79820593, 0.35652261, 0.97361289, 0.49218272]),
 array([0.22222346, 0.25200656, 0.0286736 ]))

In [4]:
outer_product = np.einsum('i,j->ij', A, B) # i,j --> Free indices
outer_product

array([[0.09940156, 0.11272368, 0.01282583],
       [0.17738008, 0.20115313, 0.02288744],
       [0.07922769, 0.08984604, 0.01022279],
       [0.21635962, 0.24535684, 0.02791699],
       [0.10937454, 0.12403328, 0.01411265]])

<p align="center">
<img src="../images/Einsum 2.png" style="width:450px;height:250px;">
</p>

In [5]:
x = np.ones(3)
sum_x = np.einsum('i->', x)
sum_x

3.0

In [8]:
y = np.ones(3)*2
print(x,y)
inner_product = np.einsum('i,i->',x,y)
inner_product

[1. 1. 1.] [2. 2. 2.]


6.0

## Permutation of Tensors

In [10]:
x = np.ones((5,4,3))
np.einsum('ijk->kji',x).shape # Will transpose

(3, 4, 5)

In [13]:
x = torch.rand((2,3))
x

tensor([[0.5580, 0.2671, 0.3252],
        [0.9512, 0.4847, 0.1854]])

In [14]:
torch.einsum('ij->ji', x) # Transpose

tensor([[0.5580, 0.9512],
        [0.2671, 0.4847],
        [0.3252, 0.1854]])

## Summation

In [16]:
torch.einsum('ij->',x) 
# Returns sum of all the 6 elements
# Happens as we did not specify the output dimension, hence all input dimensions are summation indices

tensor(2.7716)

## Column Sum

In [17]:
torch.einsum('ij->j',x)

tensor([1.5092, 0.7518, 0.5106])

## Row Sum

In [18]:
torch.einsum('ij->i',x)

tensor([1.1503, 1.6213])

## Matrix-Vector Multiplication

In [19]:
v = torch.rand((1,3)) # Row vector

For matrix multiplication between x (2,3) and v(1,3), we would have done $xv^T$

In [20]:
torch.einsum('ij,kj->ik', x, v) # It has reshaped by itself

tensor([[0.8530],
        [1.2412]])

## Matrix-Matrix Multiplication
Lets do x (2,3) with x(2,3) itself

In [21]:
torch.einsum('ij,kj->ik',x,x) # For (2,3) x (3,2)

tensor([[0.4885, 0.7205],
        [0.7205, 1.1741]])

In [22]:
torch.einsum('ij,ik->jk',x,x) # For (3,2) x (2,3)

tensor([[1.2161, 0.6101, 0.3578],
        [0.6101, 0.3063, 0.1767],
        [0.3578, 0.1767, 0.1401]])

## Dot product 1st row with 1st row of matrix

In [26]:
x[0]

tensor([0.5580, 0.2671, 0.3252])

In [25]:
# Dot product of 1st row of x with itself
torch.einsum("i,i->",x[0],x[0])

tensor(0.4885)

## Dot product with matrix

In [27]:
torch.einsum("ij,ij->",x,x)

tensor(1.6625)

In [33]:
sum(sum(x*x)) # Sum of all the elements in the element wise multiplied x with x

tensor(1.6625)

## Element-wise multiplication (Hadamard Product)

In [34]:
torch.einsum("ij,ij->ij",x,x)

tensor([[0.3114, 0.0713, 0.1057],
        [0.9047, 0.2350, 0.0344]])

In [35]:
x*x

tensor([[0.3114, 0.0713, 0.1057],
        [0.9047, 0.2350, 0.0344]])

## Outer Product

In [41]:
a = torch.rand((3))
b = torch.rand((5))

In [37]:
torch.einsum("i,j->ij",a,b), torch.einsum("i,j->ji",a,b)

(tensor([[0.0597, 0.0088, 0.0661, 0.0332, 0.1593],
         [0.0339, 0.0050, 0.0376, 0.0189, 0.0906],
         [0.2717, 0.0402, 0.3008, 0.1512, 0.7250]]),
 tensor([[0.0597, 0.0339, 0.2717],
         [0.0088, 0.0050, 0.0402],
         [0.0661, 0.0376, 0.3008],
         [0.0332, 0.0189, 0.1512],
         [0.1593, 0.0906, 0.7250]]))

## Batch Matrix Multiplication

In [46]:
a = torch.rand((3,2,5))
b = torch.rand((3,5,3))

In [47]:
torch.einsum("ijk,ikl->ijl",a,b), torch.einsum("ijk,ikl->ijl",a,b).shape

(tensor([[[1.9635, 0.8899, 2.4917],
          [1.3289, 0.8420, 1.6077]],
 
         [[1.5425, 1.1133, 0.9360],
          [1.9515, 1.4296, 1.2584]],
 
         [[1.6735, 0.9525, 1.2178],
          [1.0512, 0.8327, 0.7687]]]),
 torch.Size([3, 2, 3]))

In [48]:
torch.bmm(a,b), torch.bmm(a,b).shape # bmm: Batch Matrix Multiplication

(tensor([[[1.9635, 0.8899, 2.4917],
          [1.3289, 0.8420, 1.6077]],
 
         [[1.5425, 1.1133, 0.9360],
          [1.9515, 1.4296, 1.2584]],
 
         [[1.6735, 0.9525, 1.2178],
          [1.0512, 0.8327, 0.7687]]]),
 torch.Size([3, 2, 3]))

## Matrix Diagonal

In [50]:
x = torch.rand((3,3))
x

tensor([[0.2410, 0.0923, 0.4613],
        [0.1305, 0.0871, 0.5266],
        [0.3240, 0.7481, 0.3638]])

In [51]:
torch.einsum('ii->i',x) # ii => Diagonal values

tensor([0.2410, 0.0871, 0.3638])

## Matrix Trace

In [52]:
torch.einsum('ii->',x)

tensor(0.6919)

In [53]:
sum(torch.einsum('ii->i',x))

tensor(0.6919)

There are way more **advanced** operations which can be done easily using `einsum`